#학습과 검증을 실시

In [10]:
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

#utils를 포함한 필요한 패키지를 import

In [11]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용장치：", device)

사용장치： cuda:0


#사용중인 장치는 GPU

In [13]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


rootpath = "/content/drive/MyDrive/Colab Notebooks/2_objectdetection/data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)
input_size = 300

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

#SSD를 학습시킬 데이터 셋과 데이터 로더를 앞에서 구현한 파일들과 같이 작성

In [14]:
from utils.ssd_model import SSD

ssd_cfg = {
    'num_classes': 21,
    'input_size': 300,
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

net = SSD(phase="train", cfg=ssd_cfg)

vgg_weights = torch.load("./drive/MyDrive/Colab Notebooks/2_objectdetection/weights/vgg16_reducedfc.pth")
net.vgg.load_state_dict(vgg_weights)



def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치：", device)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다.')

사용 중인 장치： cuda:0
네트워크 설정 완료: 학습된 가중치를 로드했습니다.


#SSD의 설정은 이전 장과 똑같이 설정 후 GPU를 사용할 수 있는지 확인

In [15]:
from utils.ssd_model import MultiBoxLoss

criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)

#손실함수와 최적화 기법을 설정

In [16]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 중인 장치:", device)

    net.to(device)

    torch.backends.cudnn.benchmark = True

    iteration = 1
    epoch_train_loss = 0.0
    epoch_val_loss = 0.0
    logs = []

    for epoch in range(num_epochs+1):

        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()
                    print('-------------')
                    print('（val）')
                else:
                    continue


            for images, targets in dataloaders_dict[phase]:

                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(images)

                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    if phase == 'train':
                        loss.backward()

                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()

                        if (iteration % 10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    else:
                        epoch_val_loss += loss.item()

        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0
        epoch_val_loss = 0.0

        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')

#모델을 학습시키는 함수를 작성

In [17]:
num_epochs= 10
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 중인 장치: cuda:0
-------------
Epoch 1/10
-------------
（train）


/content/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


반복 10 || Loss: 16.5893 || 10iter: 627.1248 sec.
반복 20 || Loss: 12.9148 || 10iter: 514.8128 sec.
반복 30 || Loss: 10.6132 || 10iter: 499.5855 sec.
반복 40 || Loss: 8.9026 || 10iter: 500.9867 sec.
반복 50 || Loss: 8.0683 || 10iter: 502.8193 sec.
반복 60 || Loss: 8.0118 || 10iter: 495.3226 sec.
반복 70 || Loss: 8.5312 || 10iter: 491.7329 sec.
반복 80 || Loss: 7.7067 || 10iter: 496.8826 sec.
반복 90 || Loss: 8.1908 || 10iter: 502.7754 sec.
반복 100 || Loss: 7.6779 || 10iter: 498.1438 sec.


FileNotFoundError: ignored

### 학습을 진행했지만 반복 10번째에서 멈추어 버린다